In [ ]:
!git clone https://github.com/mana438/RNABERT

Cloning into 'RNABERT'...
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 80 (delta 31), reused 66 (delta 17), pack-reused 0
Receiving objects: 100% (80/80), 347.43 KiB | 4.34 MiB/s, done.
Resolving deltas: 100% (31/31), done.


In [ ]:
%cd RNABERT

[Errno 2] No such file or directory: 'RNABERT'
/hpc/group/chatterjee/srikar/storage/rna-binding/RNABERT


In [ ]:
!pip install .

Defaulting to user installation because normal site-packages is not writeable
Processing /hpc/group/chatterjee/srikar/storage/rna-binding/RNABERT
  Created wheel for alignment-C: filename=alignment_C-1.0.0-cp38-cp38-linux_x86_64.whl size=74772 sha256=29ceaf92afb3b1ba4671090c8ab296066771609d4baff2af0efbd0e2e32d5096
  Stored in directory: /tmp/pip-ephem-wheel-cache-demohdkt/wheels/d1/f2/1a/17de7ad1bc228b6a57a7cf15042be63829dd2fc686d4c28d90
Successfully built alignment-C


In [ ]:
pip install biopython==1.77

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 2.3 MB 4.4 MB/s eta 0:00:01
  Attempting uninstall: biopython
    Found existing installation: biopython 1.81
    Uninstalling biopython-1.81:
      Successfully uninstalled biopython-1.81
ERROR: Could not install packages due to an OSError: [Errno 39] Directory not empty: 'Align'

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import random
import time
import numpy as np
import torch
import torch.optim as optim
from torchvision import transforms, datasets
import copy
from Bio import SeqIO
import argparse
import datetime
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics.cluster import adjusted_rand_score
import os
import time
from sklearn.metrics import normalized_mutual_info_score, adjusted_rand_score, completeness_score, homogeneity_score
import torch.nn.functional as F
from sklearn.cluster import MiniBatchKMeans, KMeans, AgglomerativeClustering, SpectralClustering
import itertools

In [ ]:
pip install attrdict

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from utils.bert import get_config, BertModel, set_learned_params, BertForMaskedLM, visualize_attention, show_base_PCA, fix_params
from module import Train_Module
from dataload import DATA, MyDataset
import alignment_C as Aln_C

In [ ]:
pip show biopython

Name: biopython
Version: 1.77
Summary: Freely available tools for computational molecular biology.
Home-page: https://biopython.org/
Author: The Biopython Contributors
Author-email: biopython@biopython.org
License: UNKNOWN
Location: /hpc/home/vk93/.local/lib/python3.8/site-packages
Requires: numpy
Required-by: dscript, bio, ankh
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd

In [ ]:
attract = pd.read_csv('attract_hs.csv')

In [ ]:
fasta_content = ""

for index, row in attract.iterrows():
    fasta_content += f">{row['Gene_name']}|{row['Gene_id']}\n{row['Motif']}\n"

# Save to a fasta file
with open('rna_motif_attract.fasta', 'w') as file:
    file.write(fasta_content)


In [ ]:
!python MLM_SFP.py --pretraining bert_mul_2.pth --data_embedding rna_motif_attract.fasta --embedding_output embeddings_attract_rnabert.txt --batch 40

start...
device:  cuda
-----start-------


In [ ]:
embeddings = pd.read_table('embeddings_attract_rnabert.txt',names=['rna_motif_emb'])

In [ ]:
type(embeddings['rna_motif_emb'].iloc[0])

str

In [ ]:
import ast

In [ ]:
embeddings['rna_motif_emb'] = embeddings['rna_motif_emb'].apply(lambda x: np.array(ast.literal_eval(x)))

In [ ]:
len(embeddings)

3252

In [ ]:
len(embeddings['rna_motif_emb'].iloc[0][1]) #gives 120 parameter embedding for each rna nucleotide

120

In [ ]:
string_test = 'UGAUCAGUAUA'
print(len(string_test))

11


In [ ]:
type(embeddings)

pandas.core.frame.DataFrame

In [ ]:
attract['rna_motif_emb'] = embeddings['rna_motif_emb']

In [ ]:
attract

,Gene_name,Gene_id,Motif,RBP_sequence,rna_motif_emb
0,A1CF,ENSG00000148584,UGAUCAGUAUA,MESNHKSGDGLSGTQKEAALRALVQRTGYSLVQENGQRKYGGPPPG...,"[[0.02515462040901184, -0.1693081259727478, 1...."
1,A1CF,ENSG00000148584,AUAAUUA,MESNHKSGDGLSGTQKEAALRALVQRTGYSLVQENGQRKYGGPPPG...,"[[-0.005481339991092682, -0.008752591907978058..."
2,A1CF,ENSG00000148584,UUAAUUA,MESNHKSGDGLSGTQKEAALRALVQRTGYSLVQENGQRKYGGPPPG...,"[[-0.006157027557492256, -0.04564734920859337,..."
3,A1CF,ENSG00000148584,AUAAUUG,MESNHKSGDGLSGTQKEAALRALVQRTGYSLVQENGQRKYGGPPPG...,"[[0.0016195997595787048, 0.0697508156299591, 1..."
4,A1CF,ENSG00000148584,UUAAUUG,MESNHKSGDGLSGTQKEAALRALVQRTGYSLVQENGQRKYGGPPPG...,"[[0.02757852151989937, 0.006868686527013779, 3..."
...,...,...,...,...,...
3247,SRSF1,ENSG00000136450,CGGCGGU,MSGGGVIRGPAGNNDCRIYVGNLPPDIRTKDIEDVFYKYGAIRDID...,"[[0.03172174096107483, -0.21967869997024536, -..."
3248,SRSF2,ENSG00000161547,GAAAGGAGA,MSYGRPPPDVEGMTSLKVDNLTYRTSPDTLRRVFEKYGRVGDVYIP...,"[[0.013123728334903717, -0.012446027249097824,..."
3249,HNRNPAB,ENSG00000197451,AUAGCA,MSEAGEEQPMETTGATENGHEAVPEASRGRGWTGAAAGAGGATAAP...,"[[-0.0034322012215852737, 0.000896155834197998..."
3250,HNRNPA1,ENSG00000135486,UAGG,MSKSESPKEPEQLRKLFIGGLSFETTDESLRSHFEQWGTLTDCVVM...,"[[0.11391925066709518, -0.08793018758296967, 7..."


In [ ]:
attract.to_csv('attract_with_motif_emb.csv',index=False)